<a href="https://colab.research.google.com/github/KimSiGyum/Python/blob/main/mediapipe(%ED%95%99%EC%83%9D%EC%9A%A9)_%EB%A7%88%EC%A7%80%EB%A7%89%EA%B3%BC%EC%A0%9C_%EC%98%88%EC%8B%9C%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from mediapipe.tasks.python import vision
import threading
import cv2
import time

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

In [ ]:
# 영상에서 손을 감지하고 결과를 처리하는 클래스
class HandLandmarkerAndResult:
    def __init__(self):
        base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
        self.options = vision.HandLandmarkerOptions(
            base_options=base_options,
            running_mode=vision.RunningMode.LIVE_STREAM,
            num_hands=2,
            result_callback=self.process_result
        )
        self.hand_landmarker = vision.HandLandmarker.create_from_options(self.options)
        self.result = None
        self.event = threading.Event()

    def process_result(self, result: vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
        """감지 결과를 처리하는 콜백 함수"""
        self.result = result
        self.event.set()  # 결과가 준비되었음을 알림

    def detect_async(self, frame, timestamp_ms):
        """비동기적으로 손을 감지하고 결과를 self.result에 저장합니다."""

        # mediapipe Image 객체로 변환
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        # 비동기적으로 감지 실행
        self.hand_landmarker.detect_async(mp_image, timestamp_ms)

    def close(self):
        self.hand_landmarker.close()


In [ ]:
# 손에 랜드마크 그리는 함수
# 참고 https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

  return annotated_image

In [ ]:
# HandLandmarker 객체 생성하기
detector = HandLandmarkerAndResult()

# 캠 불러오기
cap = cv2.VideoCapture(0)

while True:  # 카메라 동작하는 동안 계속하여 반복하기
    ret, frame = cap.read()  # 비디오 스트림 (카메라 캡처)
    if not ret:  # 프레임을 가져오지 못하면 종료
        break
    frame = cv2.flip(frame, 1)  # 화면 좌우 반전

    # 비동기적으로 손 감지
    timestamp_ms = cv2.getTickCount() / cv2.getTickFrequency() * 1000  # 타임스탬프 생성
    detector.detect_async(frame, int(timestamp_ms))

    # 결과를 기다림
    detector.event.wait()

    # 결과 처리
    detection_result = detector.result

    # 손 감지 결과가 있으면 이미지에 그리기
    if detection_result:
        frame = draw_landmarks_on_image(frame, detection_result)
        frame = put_handedness_on_image(frame, detection_result)
        frame = put_index_finger_on_image(frame, detection_result)
        frame = count_fingers_raised(frame, detection_result)
        cv2.imshow('frame', frame)
    else:
        # 손 감지 결과가 없으면 원본 이미지 출력
        cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()  # 카메라 해제
cv2.destroyAllWindows()


In [ ]:
# 왼손, 오른손 표기
SIZE = 1
THICKNESS = 2
COLOR = (88, 205, 54)

def put_handedness_on_image(rgb_image, detection_result):
    handedness_list = detection_result.handedness
    for handedness in handedness_list:
        cv2.putText(rgb_image, handedness[0].category_name, (0, 50), cv2.FONT_HERSHEY_DUPLEX, SIZE, COLOR, THICKNESS)

    return rgb_image

In [ ]:
#왼손, 오른손 표기 (인덱스를 사용해서 풀기)
SIZE = 1
THICKNESS = 2
COLOR = (88, 205, 54)

def put_handedness_on_image(rgb_image, detection_result):



    return rgb_image

In [ ]:
#두 번째 손가락 표기하기
SIZE = 1
THICKNESS = 2
COLOR = (88, 205, 54)

def put_index_finger_on_image(rgb_image, detection_result):
    landmarks_list = detection_result.hand_landmarks
    for landmarks in landmarks_list:
        x = int(landmarks[8].x * rgb_image.shape[1])
        y = int(landmarks[8].y * rgb_image.shape[0])
        cv2.putText(rgb_image, 'index_tip', (x, y), cv2.FONT_HERSHEY_DUPLEX, SIZE, COLOR, THICKNESS)

    return rgb_image

In [ ]:
# 손가락이 몇 개 올라와 있는지 세는 함수
SIZE = 1
THICKNESS = 2
COLOR = (88, 205, 54)

def count_fingers_raised(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    count = 0

    for hand_landmarks in hand_landmarks_list:
        for i in range(8, 21, 4): # 검지, 중지, 약지, 새끼 손가락의 경우
            # 손끝의 y좌표와 나머지 관절의 y좌표 중 최소값을 비교하여 손끝이 더 위라면 카운트
            if hand_landmarks[i].y < min(hand_landmarks[i-1].y, hand_landmarks[i-2].y, hand_landmarks[i-3].y):
                count += 1
        # 엄지 손가락의 경우
        if hand_landmarks[4].x > hand_landmarks[0].x: # 엄지 손끝의 x좌표와 손바닥 좌표를 비교하여 엄지가 더 오른쪽에 있고
            # 엄지 손끝의 x좌표와 나머지 관절의 x좌표 중 최대값을 비교하여 엄지가 가장 오른쪽에 있다면 카운트
            if hand_landmarks[4].x > max(hand_landmarks[i-1].y, hand_landmarks[i-2].y, hand_landmarks[i-3].y):
                count += 1
        else: # 반대로 엄지 손끝의 x좌표와 손바닥 좌표를 비교하여 엄지가 더 왼쪽에 있고
            # 엄지 손끝의 x좌표와 나머지 관절의 x좌표 중 최소값을 비교하여 엄지가 가장 왼쪽에 있다면 카운트
            if hand_landmarks[4].x < min(hand_landmarks[i-1].y, hand_landmarks[i-2].y, hand_landmarks[i-3].y):
                count += 1

    cv2.putText(rgb_image, str(count), (0, 100), cv2.FONT_HERSHEY_DUPLEX, SIZE, COLOR, THICKNESS)

    return rgb_image